# Kalman arbitrage

In [61]:
import numpy as np
import pandas as pd
import theano 
import theano.tensor as T
import matplotlib.pyplot as plt

%matplotlib inline

In [62]:
import time
import tqdm
import datetime
import glob

In [63]:
from sklearn.linear_model import LinearRegression
from itertools import *

### Prepare data

In [22]:
path = 'Q:/MSK1_QR/Strategies/Es/pairTradingCheckFullFrom2013/'
universe = list(map(lambda s: s[54:-4], glob.glob(path+'*.csv')))
data = pd.DataFrame()
for sym in tqdm.tqdm(universe):
    sym_data = pd.read_csv(path + sym + '.csv', usecols=['Date', 'Time', 'Close'], parse_dates=[['Date', 'Time']],\
                            index_col='Date_Time')
    data = pd.concat([data, sym_data], axis=1)


100%|████████████████████████████████████████| 114/114 [00:31<00:00,  3.04it/s]


In [28]:
data.columns = universe
data.ffill(inplace=True)
data.head(1)

,AM,APA,APC,AR,ARLP,BHI,BP,BPL,CEO,CEQP,...,VET,VLO,WES,WGP,WMB,WNR,WPZ,XEC,XOM,YPF
Date_Time,,,,,,,,,,,,,,,,,,,,,
2011-01-03 09:45:00,NaN,114.92,73.19,NaN,26.2875,53.69,31.435,39.3575,206.886036,-297.75,...,NaN,15.26,14.15,NaN,14.96375,0.59,12.523187,87.46,57.35,46.272106


In [37]:
time_index = data.loc['2012':].index

In [48]:
time_index_dates = np.unique(list(map(lambda t: t.date(), time_index)))
time_index_date_ends = list(map(lambda d: datetime.datetime(d.year, d.month, d.day, 15, 59, 0), time_index_dates))

In [111]:
for i, t in tqdm.tqdm(enumerate(time_index_date_ends)):
    data_slice = data.loc[:t].tail(int(6.5 * 4 * 125))
    data_slice.dropna(axis=1, inplace=True)
    y_cols = [data_slice.columns[tup[0]] for tup in combinations(range(data_slice.shape[1]), 2)]
    x_cols = [data_slice.columns[tup[1]] for tup in combinations(range(data_slice.shape[1]), 2)]
    data_slice.to_csv('D:/data/storage/slices/data_slice_{}.csv'.format(t.date()))
    pairs =     pd.DataFrame(np.vstack([y_cols, x_cols]).T, columns=['symbol_y', 'symbol_x'])\
            .to_csv('D:/data/storage/slices/pairs_{}.csv'.format(t.date()), index_label='id')

1367it [10:54,  1.88it/s]
